In [1]:
import csv
import math
import os
import cv2
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw, ImageEnhance
import numpy as np
import pandas as pd
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.layers import Conv2D, Reshape, DepthwiseConv2D, ZeroPadding2D
from tensorflow.keras.utils import Sequence
from tensorflow.keras.backend import epsilon

from sklearn.model_selection import train_test_split

DATA_DIR = "images"

# 0.35, 0.5, 0.75, 1.0, 1.3, 1.4
ALPHA = 1.3

# 96, 128, 160, 192, 224
IMAGE_SIZE = 299

EPOCHS = 200
BATCH_SIZE = 24
PATIENCE = 50

MULTI_PROCESSING = True
THREADS = 12

data = pd.read_csv('training.csv')
train, test = train_test_split(data, test_size=0.1, random_state=123)

train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

train.to_csv('train.csv', index=False)
test.to_csv('validation.csv', index=False)

TRAIN_CSV = "train.csv"
VALIDATION_CSV = "validation.csv"

In [2]:

class DataGenerator(Sequence):

    def __init__(self, csv_file,rnd_rescale=True, rnd_multiply=True, rnd_color=True, rnd_crop=True, rnd_flip=False,
                 batch_size = BATCH_SIZE):
        self.paths = []
        self.coords = []
        self.batch_size = batch_size
        self.rnd_rescale = rnd_rescale
        self.rnd_multiply = rnd_multiply
        self.rnd_color = rnd_color
        self.rnd_crop = rnd_crop
        self.rnd_flip = rnd_flip

        with open(csv_file, "r") as file:
            self.coords = np.zeros((sum(1 for line in file)-1, 4))
            
        df = pd.read_csv(csv_file)

        for index, row in df.iterrows():
            path = os.path.join(DATA_DIR, row['image_name'])
            x1 = row['x1']
            x2 = row['x2']
            y1 = row['y1']
            y2 = row['y2']

            img = Image.open(path)
            width, height = img.size

            self.coords[index, 0] = x1
            self.coords[index, 1] = y1
            self.coords[index, 2] = x2 
            self.coords[index, 3] = y2 

            self.paths.append(path)

    def __len__(self):
        return math.ceil(len(self.coords) / self.batch_size)

    def __getitem__(self, idx):
        batch_paths = self.paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_coords = self.coords[idx * self.batch_size:(idx + 1) * self.batch_size].copy()

        batch_images = np.zeros((len(batch_paths), IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.float32)
        for i, f in enumerate(batch_paths):
            img = Image.open(f)
            x0,y0,x1,y1 = batch_coords[i]
            if self.rnd_rescale:
                old_width = img.width
                old_height = img.height

                rescale = np.random.uniform(low=0.6, high=1.4)
                new_width = int(old_width * rescale)
                new_height = int(old_height * rescale)

                img = img.resize((new_width, new_height))

                x0 *= new_width / old_width
                y0 *= new_height / old_height
                x1 *= new_width / old_width
                y1 *= new_height / old_height

            if self.rnd_crop:
                start_x = np.random.randint(0, high=np.floor(0.15 * img.width))
                stop_x = img.width - np.random.randint(start_x, high=np.floor(0.15 * img.width))
                start_y = np.random.randint(0, high=np.floor(0.15 * img.height))
                stop_y = img.height - np.random.randint(start_y, high=np.floor(0.15 * img.height))

                img = img.crop((start_x, start_y, stop_x, stop_y))

                x0 = max(x0 - start_x, 0)
                y0 = max(y0 - start_y, 0)
                x1 = min(x1 - start_x, img.width)
                y1 = min(y1 - start_y, img.height)

                if np.abs(x1 - x0) < 5 or np.abs(y1 - y0) < 5:
                    print("\nWarning: cropped too much (obj width {}, obj height {}, img width {}, img height {})\n".format(x1 - x0, y1 - y0, img.width, img.height))

            if self.rnd_flip:
                elem = np.random.choice([0, 90, 180, 270, 1423, 1234])
                if elem % 10 == 0:
                    x = x0 - img.width / 2
                    y = y0 - img.height / 2

                    x0 = img.width / 2 + x * np.cos(np.deg2rad(elem)) - y * np.sin(np.deg2rad(elem))
                    y0 = img.height / 2 + x * np.sin(np.deg2rad(elem)) + y * np.cos(np.deg2rad(elem))

                    x = x1 - img.width / 2
                    y = y1 - img.height / 2

                    x1 = img.width / 2 + x * np.cos(np.deg2rad(elem)) - y * np.sin(np.deg2rad(elem))
                    y1 = img.height / 2 + x * np.sin(np.deg2rad(elem)) + y * np.cos(np.deg2rad(elem))

                    img = img.rotate(-elem)
                else:
                    if elem == 1423:
                        img = img.transpose(Image.FLIP_TOP_BOTTOM)
                        y0 = img.height - y0
                        y1 = img.height - y1

                    elif elem == 1234:
                        img = img.transpose(Image.FLIP_LEFT_RIGHT)
                        x0 = img.width - x0
                        x1 = img.width - x1

            image_width = img.width
            image_height = img.height

            tmp = x0
            x0 = min(x0, x1)
            x1 = max(tmp, x1)

            tmp = y0
            y0 = min(y0, y1)
            y1 = max(tmp, y1)

            x0 = max(x0, 0)
            y0 = max(y0, 0)

            y0 = min(y0, image_height)
            x0 = min(x0, image_width)
            y1 = min(y1, image_height)
            x1 = min(x1, image_width)

            if self.rnd_color:
                enhancer = ImageEnhance.Color(img)
                img = enhancer.enhance(np.random.uniform(low=0.5, high=1.5))

                enhancer2 = ImageEnhance.Brightness(img)
                img = enhancer.enhance(np.random.uniform(low=0.7, high=1.3))

            img = img.resize((IMAGE_SIZE, IMAGE_SIZE))
            img = img.convert('RGB')
            pil_img = img
            img = np.array(img, dtype=np.float32)
            pil_img.close()
            if self.rnd_multiply:
                img[...,0] = np.floor(np.clip(img[...,0] * np.random.uniform(low=0.8, high=1.2), 0.0, 255.0))
                img[...,1] = np.floor(np.clip(img[...,1] * np.random.uniform(low=0.8, high=1.2), 0.0, 255.0))
                img[...,2] = np.floor(np.clip(img[...,2] * np.random.uniform(low=0.8, high=1.2), 0.0, 255.0))

            batch_images[i] = preprocess_input(img.copy())
            
            batch_coords[i, 0] = x0 * IMAGE_SIZE / image_width
            batch_coords[i, 1] = y0 * IMAGE_SIZE / image_height
            batch_coords[i, 2] = (x1 - x0) * IMAGE_SIZE / image_width
            batch_coords[i, 3] = (y1 - y0) * IMAGE_SIZE / image_height 

        return batch_images, batch_coords
            
        return np.concatenate((batch_images, batch_images_rotate90)), np.concatenate((batch_coords, batch_coords_rotate90))
class Validation(Callback):
    def __init__(self, generator):
        self.generator = generator

    def on_epoch_end(self, epoch, logs):
        mse = 0
        intersections = 0
        unions = 0

        for i in range(len(self.generator)):
            batch_images, gt = self.generator[i]
            pred = self.model.predict_on_batch(batch_images)
            mse += np.linalg.norm(gt - pred, ord='fro') / pred.shape[0]

            pred = np.maximum(pred, 0)

            diff_width = np.minimum(gt[:,0] + gt[:,2], pred[:,0] + pred[:,2]) - np.maximum(gt[:,0], pred[:,0])
            diff_height = np.minimum(gt[:,1] + gt[:,3], pred[:,1] + pred[:,3]) - np.maximum(gt[:,1], pred[:,1])
            intersection = np.maximum(diff_width, 0) * np.maximum(diff_height, 0)

            area_gt = gt[:,2] * gt[:,3]
            area_pred = pred[:,2] * pred[:,3]
            union = np.maximum(area_gt + area_pred - intersection, 0)

            intersections += np.sum(intersection * (union > 0))
            unions += np.sum(union)

        iou = np.round(intersections / (unions + epsilon()), 4)
        logs["val_iou"] = iou

        mse = np.round(mse, 4)
        logs["val_mse"] = mse

        print(" - val_iou: {} - val_mse: {}".format(iou, mse))

def create_irv2(trainable=False):
    model = InceptionResNetV2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, weights=None)

    # to freeze layers
    for layer in model.layers:
        layer.trainable = False

    x = model.layers[-1].output
    x = Conv2D(512, kernel_size=4, name="coords1", activation='relu')(x)
    x = Conv2D(256, kernel_size=3, name="coords2", activation='relu')(x)
    x = Conv2D(4, kernel_size=3, name="coords3", activation='relu')(x)
    x = Reshape((4,))(x)

    return Model(inputs=model.input, outputs=x)

def create_mnv2(trainable=False):
    model = MobileNetV2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, alpha=ALPHA, weights=None)

    # to freeze layers
    for layer in model.layers:
        layer.trainable = False

    x = model.layers[-1].output
    x = Conv2D(256, kernel_size=3, name="coords1", activation='relu')(x)
    x = Conv2D(4, kernel_size=5, name="coords2", activation='relu')(x)
    x = Reshape((4,))(x)

    return Model(inputs=model.input, outputs=x)


In [5]:
mnv2 = create_mnv2()
mnv2.load_weights('mobilenetV2-no-weights-all_layers-0.91.h5')

irv2 = create_irv2()
irv2.load_weights('inception_resnetV2-no_weights-freeze-0.91.h5')




ResourceExhaustedError: OOM when allocating tensor of shape [] and type float
	 [[node Conv1_2/kernel/Initializer/random_uniform/sub (defined at /usr/local/lib/python3.5/dist-packages/keras_applications/mobilenet_v2.py:347)  = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [] values: 0.24902913>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'Conv1_2/kernel/Initializer/random_uniform/sub', defined at:
  File "/usr/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/us

In [60]:
sample = pd.read_csv("train.csv")
sample.head()

,image_name,x1,x2,y1,y2
0,1474724014247DSC07926.png,258,424,96,381
1,JPEG_20160623_170956_1000885486700.png,293,583,134,359
2,147185868090020160818_111244.png,112,478,2,452
3,JPEG_20160513_115642_1000775593523.png,83,589,168,398
4,JPEG_20161118_133812_1000159834754.png,172,519,0,480


In [ ]:

for filename in glob.glob(IMAGES)[8000:8010]:
    unscaled = cv2.imread(filename)
    image_height, image_width, _ = unscaled.shape

    image = cv2.resize(unscaled, (IMAGE_SIZE, IMAGE_SIZE))
    feat_scaled = preprocess_input(np.array(image, dtype=np.float32))

    region = model.predict(x=np.array([feat_scaled]))[0]

    x0 = int(region[0] * image_width / IMAGE_SIZE)
    y0 = int(region[1] * image_height / IMAGE_SIZE)

    x1 = int((region[0] + region[2]) * image_width / IMAGE_SIZE)
    y1 = int((region[1] + region[3]) * image_height / IMAGE_SIZE)

    cv2.rectangle(unscaled, (x0, y0), (x1, y1), (0, 255, 0), 2)
    plt.imshow(unscaled)
    plt.show()

In [23]:
sample = pd.read_csv("test.csv")
sample.head()

,image_name,x1,x2,y1,y2
0,1474723840903DSC08089.png,NaN,NaN,NaN,NaN
1,1473231475010DeeplearnS11276.png,NaN,NaN,NaN,NaN
2,JPEG_20161205_135307_1000155917326.png,NaN,NaN,NaN,NaN
3,JPEG_20160711_123440_1000518778437.png,NaN,NaN,NaN,NaN
4,JPEG_20160803_115329_100034020722.png,NaN,NaN,NaN,NaN


In [24]:
for index, row in sample.iterrows():
    
    unscaled = cv2.imread('images/'+row['image_name'])
    image_height, image_width, _ = unscaled.shape
    
    IMAGE_SIZE = 299
    
    image = cv2.resize(unscaled, (IMAGE_SIZE, IMAGE_SIZE))
    feat_scaled = preprocess_input(np.array(image, dtype=np.float32))

    region = irv2.predict(x=np.array([feat_scaled]))[0]

    x1a = (region[0] * image_width / IMAGE_SIZE)
    y1a = (region[1] * image_height / IMAGE_SIZE)

    x2a = ((region[0] + region[2]) * image_width / IMAGE_SIZE)
    y2a = ((region[1] + region[3]) * image_height / IMAGE_SIZE)
    
    IMAGE_SIZE = 224

    image = cv2.resize(unscaled, (IMAGE_SIZE, IMAGE_SIZE))
    feat_scaled = preprocess_input(np.array(image, dtype=np.float32))

    region = mnv2.predict(x=np.array([feat_scaled]))[0]

    x1b = (region[0] * image_width / IMAGE_SIZE)
    y1b = (region[1] * image_height / IMAGE_SIZE)

    x2b = ((region[0] + region[2]) * image_width / IMAGE_SIZE)
    y2b = ((region[1] + region[3]) * image_height / IMAGE_SIZE)


    sample.iloc[index,1] = int((x1a + x1b)/2)
    sample.iloc[index,2] = int((x2a + x2b)/2)
    sample.iloc[index,3] = int((y1a + y1b)/2)
    sample.iloc[index,4] = int((y2a + y2b)/2)

sample.head()

,image_name,x1,x2,y1,y2
0,1474723840903DSC08089.png,231.0,440.0,88.0,408.0
1,1473231475010DeeplearnS11276.png,79.0,570.0,141.0,349.0
2,JPEG_20161205_135307_1000155917326.png,140.0,494.0,53.0,437.0
3,JPEG_20160711_123440_1000518778437.png,216.0,458.0,97.0,417.0
4,JPEG_20160803_115329_100034020722.png,127.0,501.0,45.0,428.0


In [25]:
sample.to_csv('prediction.csv', index=False)

In [11]:
len(model.layers)

784

In [8]:
model.compile(loss="mean_absolute_error", optimizer="adadelta", metrics=['accuracy'])

In [15]:
validation_datagen = Validation(generator=DataGenerator(VALIDATION_CSV, batch_size = 64))